In [1]:
import nltk
from nltk.corpus import stopwords
import pandas as pd

nltk.download('stopwords')
stop = stopwords.words('english')

df_text = pd.read_csv('rnews_apple.txt', sep=': \"', header=None, names=['date', 'text'], dtype='str', engine='python')
df_text['text_without_stopwords'] = df_text['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
print(df_text[:4])


       date                                               text  \
0  01012013              Armed robbers hit Paris Apple store",   
1  01022013  UPDATE 1-Judge rejects part of Apple App Store...   
2  01042013  US STOCKS-Data helps pace Wall St higher, but ...   
3  01062013  Riches in niches: U.S. cops, in-flight movies ...   

                              text_without_stopwords  
0              Armed robbers hit Paris Apple store",  
1  UPDATE 1-Judge rejects part Apple App Store su...  
2  US STOCKS-Data helps pace Wall St higher, Appl...  
3  Riches niches: U.S. cops, in-flight movies may...  


[nltk_data] Downloading package stopwords to /Users/apple/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('all_stocks_5yr.csv')
df = df.loc[(df['Name'] == "AAPL")]
df = df.drop('volume', 1)
df = df.drop('high', 1)
df = df.drop('low', 1)
df = df.drop('Name', 1)
print('df.shape',df.shape)
print('df.shape',df[:3])
df = df.sort_values('date').reset_index(drop=True)

df['next_day_open'] = df['open'].shift(-1)
df['next_day_close'] = df['close'].shift(-1)
df = df.dropna()
df = df.sort_values('next_day_close').reset_index(drop=True)
print('df.shape',df.shape)
print('df.shape',df[:3])

df.shape (1259, 3)
df.shape             date     open    close
1259  2013-02-08  67.7142  67.8542
1260  2013-02-11  68.0714  68.5614
1261  2013-02-12  68.5014  66.8428
df.shape (1258, 5)
df.shape          date     open    close  next_day_open  next_day_close
0  2013-04-18  57.8557  56.0071        55.4242         55.7899
1  2013-04-17  60.0385  57.5428        57.8557         56.0071
2  2013-06-26  57.6999  56.8671        57.0357         56.2542


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
import datetime
from tqdm import tqdm
import copy

class MainRNN():
    def __init__(self):
        self.data_time_range=1
        self.seq_size=self.data_time_range
        self.hidden_layer=1
        self.output_feature_size=1
        self.epochs=2000
        self.batch_size=2

    def get_formated_data(self):
                
        count_vectorizer = CountVectorizer(decode_error='ignore',binary=False)
        X_text = count_vectorizer.fit_transform(df_text['text_without_stopwords'])
        X_text = X_text.toarray()
        
        
        
        print(df[:3])

        X = []
        Y = []
        for index, row in tqdm(df.iterrows()):
            row_data_array = row.values[1:-1]
            row_day = row['date'].split('-')[2]
            row_mon = row['date'].split('-')[1]
            row_year = row['date'].split('-')[0]
            text_index = df_text[(df_text['date'] == row_mon + row_day + row_year)].index.values.astype(int)
            if len(text_index) > 0:
                text_index = text_index[0]

                if (text_index) >= 0: 
                    row_data_array = np.append(row_data_array, np.array(X_text[text_index], dtype=np.int))
                else:
                    _, shape_for_use = np.array(X_text).shape
                    row_data_array = np.append(row_data_array, np.zeros(shape_for_use, dtype=np.int))
            else: 
                _, shape_for_use = np.array(X_text).shape
                row_data_array = np.append(row_data_array, np.zeros(shape_for_use, dtype=np.int))
                
            X.append([row_data_array])
            Y.append([row['next_day_close']])
        
        print(X[1], Y[1])
        print('before X.shape', len(X))
        print('before Y.shape', len(Y))
        
        return np.array(X), np.array(Y)


    def process_train(self, X, Y):

        # get shape X (N, T, D)
        X_sample_size, X_seq_size, X_features_size = X.shape
        print('X_features_size', X_features_size)

        # get shape Y (K)
        Y_sample_size = Y.shape

        # init weight and bias
        weights = tf.Variable(tf.random_normal([self.hidden_layer, self.output_feature_size]))
        biases = tf.Variable(tf.random_normal([self.output_feature_size]))

        # placeholder for graph input
        tfX = tf.placeholder(tf.float32, shape=[None, X_seq_size, X_features_size], name='inputX')
        tfY = tf.placeholder(tf.float32, shape=[None, self.output_feature_size], name='inputY')

        # transposeX
        tfX = tf.transpose(tfX, [1, 0, 2])

        # define lstm cell
        lstmCell = tf.nn.rnn_cell.BasicLSTMCell(self.hidden_layer)

        # create RNN unit
        outputs, states = tf.nn.dynamic_rnn(cell=lstmCell, inputs=tfX, dtype=tf.float32)

        # get rnn output
        outputs = tf.stack(outputs)

        # transpose output back
        outputs = tf.transpose(outputs, [1, 0, 2])
        # outputs = tf.reshape(outputs, [outputs.get_shape()[-1], self.hidden_layer])

        # Hack to build the indexing and retrieve the right output.
        # Start indices for each sample
        index = tf.range(0, tf.shape(outputs)[0]) * X_seq_size + (X_seq_size - 1)
        # Indexing
        outputs = tf.gather(tf.reshape(outputs, [-1, self.hidden_layer]), index)

        # prediction
        prediction = tf.matmul(outputs, weights) + biases
        label = tfY

        # cost function
        loss = tf.reduce_mean(tf.square(prediction - tfY))

        # optimizer
        optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(loss)

        # cost[] and accuracies[]
        epoch_pred=[]
        epoch_lab=[]
        iterl=[]
        

        # global init
        init = tf.global_variables_initializer()

        first=False

        # start training
        with tf.Session() as sess:
            saver = tf.train.Saver()
            sess.run(init)

            for epoch in range(self.epochs):
                costs=[]
                print('===== EPOCH ======: ', epoch)
                if not first:
                    saver.restore(sess,"./trained_model")
                    first=False

#                 X, Y = shuffle(X, Y)
            
                c=0
                for batch in tqdm(range(X_sample_size - self.batch_size)):
                    batchX = X[batch:batch+self.batch_size]
                    batchY = Y[batch:batch+self.batch_size]

                    _, cost_out, prediction_out = sess.run([optimizer, loss, prediction], feed_dict={tfX: batchX.reshape(X_seq_size, self.batch_size, X_features_size), tfY: batchY.reshape(self.batch_size, self.output_feature_size)})
                    costs.append(cost_out)
                    if epoch == self.epochs-1:
                        
                        epoch_pred.append(prediction_out[1])
                        epoch_lab.append(Y[batch+1])
                        iterl.append(c)
                        c+=1

                meanse = sum(costs) / float(len(costs))
                
                print('meanse: ', meanse)
                
                saver.save(sess, "./trained_model")

            np.random.seed(0)
#             plt.plot(epochsl, epochs_costs, 'g^')
            plt.plot(iterl, epoch_pred, 'r--', iterl, epoch_lab, 'b--')
            plt.savefig('costs.png')


    def run_prediction(self):
        print('start time:', datetime.datetime.now())
        X, Y = self.get_formated_data()
        self.process_train(X, Y)
        print('end time:', datetime.datetime.now())


if __name__ == '__main__':
    MainRNN().run_prediction()


/Users/apple/anaconda/envs/HappyNewYear/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/apple/anaconda/envs/HappyNewYear/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
27it [00:00, 267.19it/s]

start time: 2019-05-10 15:18:08.302535
         date     open    close  next_day_open  next_day_close
0  2013-04-18  57.8557  56.0071        55.4242         55.7899
1  2013-04-17  60.0385  57.5428        57.8557         56.0071
2  2013-06-26  57.6999  56.8671        57.0357         56.2542


1258it [00:04, 258.41it/s]


[array([60.0385, 57.5428, 57.8557, ..., 0, 0, 0], dtype=object)] [56.0071]
before X.shape 1258
before Y.shape 1258
X_features_size 2721


/Users/apple/anaconda/envs/HappyNewYear/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


...

100%|██████████| 1256/1256 [00:02<00:00, 434.50it/s]


meanse:  939.6712006360862
===== EPOCH ======:  599
INFO:tensorflow:Restoring parameters from ./trained_model


100%|██████████| 1256/1256 [00:03<00:00, 337.01it/s]


meanse:  939.6664814690876
===== EPOCH ======:  600
INFO:tensorflow:Restoring parameters from ./trained_model


 14%|█▍        | 174/1256 [00:00<00:03, 306.18it/s]